<FONT size="5.9pt"> **(1) SPIRALGORITHM: BIOREFINERY MODEL & FOREGROUND DATABASES**
</FONT> 

This jupyter notebook demonstrates how the foreground databases were generated to perform the environmental life cycle assessment (LCA) study of the SpiralG biorefinery. The algorithm is based on the data collected at the three pilot scale facilities in the frame of the SpiralG H2020 CBE-JU project. The model is parameterised in order to compare the data for two distinct periods: 2019 (period 1) and 2021-2022 (period 2). The outputs of this code correspond to the databases to be used for LCA calculations. The databases are also available on the github page as *bw2package* files. 

# Set up the directories

The directories are included as arguments of the functions to give flexibility in "where" the data should be accessed and the results saved. The names of the folders/files can be modified (indicated by "[to be changed]"). 

In [1]:
import os
import sys

# set up the directories in which the modules to be loaded are saved [to be changed]
mdir1 = '/home/leabraud/Documents/spiralgorithm/bioref_model'
mdir2 = '/home/leabraud/Documents/spiralgorithm/bioref2lca'
mdir_list = [mdir1, mdir2]

for mdir in mdir_list:
    if mdir not in sys.path:
        sys.path.insert(0,mdir)
    
# set up the working directory in which the datasets and databases will be saved [to be changed]
wdir = '/home/leabraud/Documents/LCASpiralg'        

# set up the directory in which the recipes are stored [to be changed]
recdir = os.path.join(wdir, 'recipes')
print(recdir)

# set up the directory in which the datasets are stored [to be changed]
ddir = os.path.join(wdir, 'datasets')
print(ddir)

# set up the directory in which the ecoinvent database is stored [to be changed]
eidir = os.path.join('/home/leabraud/Desktop/databases/ecoinvent_3.6_cut-off_ecoSpold02_complete/datasets')
ei_name = 'ecoinvent_3.6_cutoff'

# set up the file names [to be changed]
recipe_name = 'recipe_bioref.yml'
param_file_name = 'parameters.xlsx'
ei_file_name = 'ecoinvent_datasets.xlsx'
lifetime_file_name = 'material_lifetime.xlsx'

/home/leabraud/Documents/LCASpiralg/recipes
/home/leabraud/Documents/LCASpiralg/datasets


# Build the parameter file

The SpiralG biorefinery model is based on parameters which need to be defined before running the functions that calculate the inputs/outputs of each unit process. The initial values of the parameters are based on measurements performed on-site at the pilot scale facilities or derived from calculations. The function "InitialParametersDict" calculate the parameters for periods 1 and 2 (e.g., productivity, phycocyanin content, phycocyanin extraction efficiency). Most of the parameters are similar for the two periods such as the number and surface of the open raceway ponds, the number of vibrating filters etc. They were included so that the life cycle inventory data could be generated for different cultivation system setups. The file created by this function is saved in the wrking directory indicated and can be modified manually if needed. Columns with new parameter values can be added to evaluate/compare different scenarios. The parameters file is then loaded into the calculation function.

In [2]:
from S0_set_parameters import InitialParametersDict

InitialParametersDict (ddir, param_file_name)

Location of the parameter file: /home/leabraud/Documents/LCASpiralg/datasets/parameters.xlsx


,tech_1,tech_2
tech_period,1,2
line,dry,dry
productivity,6.3,3.5
PC_content,12.62,11.67
PC_extraction_efficiency,92.16,92.16
dry_spangles,26.65,12.2
blue_extract,8.64,2.82
pure_PC,3.1,1.31
working_days,365,365
spangles_production_capacity,9.73,4.45


# Run the biorefinery model based on the parameter file

In [3]:
from run_bioref_model_3Aug23 import RunBiorefModel

comparison_dict, recipe = RunBiorefModel (wdir, recdir, ddir, recipe_name, param_file_name, lifetime_file_name)


 --------------
| SPIRALGORITHM |
 --------------


> PREPARATION:
  ------------

Directories set up.

Recipe loaded.

> CALCULATION:
  -----------

Biorefinery model parameters imported.

Creation of the datasets for each scenario..

Scenario tech_1
All parameters imported.

Creation of the dataset for Spirulina production in ORPs...

Dry spangles daily productivity: 6.30 g DW-eq/m2/day (accounts for the pre-processing stages)
Considering this productivity, the annual production over 365 working days would be: 9.73 ton DW-eq/year
or 10.21 ton/year at a DM content of 94.99 percent as measured in 2022.
Concentration at harvesting: 1.07 g/L
S1A0_Building_data_dict done
S1A0_Operation_data_dict done
S1A2_Filtration_data_dict done
S1A3_Dewatering_data_dict done
line is dry
S1A4_Shaping_data_dict done
S1A5_Drying_data_dict done
S1A6b_Packaging_data_dict done
S1A8_Transport_data_dict done
S1A1_Cultivation_data_dict done

Details of the biomass flows:
Amount of broth: 72.92 kg DW-eq
Amount 

The dictionary of data generated contains all the inputs/outputs (or exchanges) of each unit porcess (or activity) considered in the biorefinery. The nested dictionary is built as follows: subsystem > period > activities > inputs/outputs.

In [4]:
# quick look at the structure of the nested dictionary
print(comparison_dict.keys())
print(comparison_dict['S1'].keys())
print(comparison_dict['S1']['tech_2'].keys())

dict_keys(['S1', 'S2', 'S3', 'transport_S12', 'transport_S23', 'infrastructures', 'operation'])
dict_keys(['tech_1', 'tech_2'])
dict_keys(['S1A0Building', 'S1A0Operation', 'S1A1Cultivation', 'S1A2Filtration', 'S1A3Dewatering', 'S1A4Shaping', 'S1A5Drying', 'S1A6Packaging', 'S1A8Transport'])


In [5]:
# details of the drying activity in S1
comparison_dict['S1']['tech_1']['S1A5Drying']

{'wet_spangles': {'amount': 26.502218440576886,
  'unit': 'kg DW-eq',
  'type': 'fg_input'},
 'dry_spangles': {'amount': 26.502218440576883,
  'unit': 'kg DW-eq',
  'type': 'ref_flow'},
 'losses': {'amount': 3.552713678800501e-15,
  'unit': 'kg DW-eq',
  'type': 'losses'},
 'electricity_IT': {'amount': 300.60406567076694,
  'unit': 'kWh',
  'type': 'tech_input'},
 'water_vapor': {'amount': 52.756583625207035,
  'unit': 'L',
  'type': 'emission'}}

In [6]:
recipe

{'main_output': {'S1': {'act_name': 'S1A5Drying',
   'exc_name': 'dry_spangles',
   'scaling_factor': 28.78},
  'S2': {'act_name': 'S2A5Ultrafiltration2',
   'exc_name': 'blue_extract_UF2',
   'scaling_factor': 5.7},
  'S3': {'act_name': 'S3A4Concentration',
   'exc_name': 'concentrate',
   'scaling_factor': 3.46}},
 'wdir': '/home/leabraud/Documents/LCASpiralg',
 'resdir': '/home/leabraud/Documents/LCASpiralg/results',
 'recdir': '/home/leabraud/Documents/LCASpiralg/recipes/recipe_bioref_copy.yml'}

# Scaling of the data

The data obatined above are not scaled, they correspond to the inputs/outputs measured or calculated for period 1 and 2 for the production of 26.5 kg DW-eq of dry spaghettini in period 1 and 14.8 kg DW-eq in period 2. In order to compare the two periods, the data need to be scaled to the theoretical daily biomass production to reach the target of 10 metric tons per year (i.e. 28.78 kg DW-eq dry spaghettini). The scaling factors were applied are as follows: 
   - **S1**: scaling factor: 28.78, main output: "S1.A5.Drying"
   - **S2**: scaling factor: 5.70, main output: "S2.A5.Ultrafiltration2"
   - **S3**: scaling factor: 3.46, main output: "S3.A4.Concentration"
   - **transport**: scaling factor: 1, main output: "S123.A8.Transport"
   - **infrastructures**: scaling factor: 1, main output: "S1.A0.Building"
   - **operation**: scaling factor: 1, main output: "S1.A0.Operation"
   
**WARNING: careful not to scale the dataset twice when performing the LCA calculations.**

In [7]:
from run_bioref_model_3Aug23 import ScaleBiorefModel

comparison_dict_per_main_output, recipe = ScaleBiorefModel (comparison_dict, recipe)


> SCALING:
  -------
                          S1                    S2                 S3
act_name          S1A5Drying  S2A5Ultrafiltration2  S3A4Concentration
exc_name        dry_spangles      blue_extract_UF2        concentrate
scaling_factor         28.78                   5.7               3.46

Data scaled to the main output and exported as Excel files.
Location of the files: /home/leabraud/Documents/LCASpiralg/results


## Create the Brightway databases for LCA study

The data dictionaries generated by the SpiralG biorefinery model are used to build foreground databases for the LCA study using an Excel file mapping the outputs of the model with ecoinvent datasets. The databases are exported in the BW2Package format and as Excel files. 

In [8]:
from dict2bw import CreateForegroundDatabase

# choose a project name [to be changed]
project_name = 'lcaspiralg'

ei_datasets_file_dir = os.path.join(wdir, ei_file_name)

for subsystem in comparison_dict_per_main_output.keys():
    
    new_comparison_dict = comparison_dict_per_main_output[subsystem]
    database_name = subsystem

    CreateForegroundDatabase (wdir, project_name, ei_name, eidir, new_comparison_dict, ei_datasets_file_dir, database_name, PerSubsystem = False)


Project directory: /home/leabraud/Documents/LCASpiralg/projects
Using environment variable BRIGHTWAY2_DIR for data directory:
/home/leabraud/Documents/LCASpiralg/projects


Writing activities to SQLite3 database:



Current project: lcaspiralg
Creating default biosphere

Applying strategy: normalize_units
Applying strategy: drop_unspecified_subcategories
Applying strategy: ensure_categories_are_tuples
Applied 3 strategies in 0.00 seconds


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/13/2024 15:12:26
  Finished: 05/13/2024 15:12:26
  Total time elapsed: 00:00:00
  CPU %: 101.10
  Memory %: 1.45
Created database: biosphere3
Creating default LCIA methods

Applying strategy: normalize_units
Applying strategy: set_biosphere_type
Applying strategy: fix_ecoinvent_38_lcia_implementation
Applying strategy: drop_unspecified_subcategories
Applying strategy: link_iterable_by_fields
c4c908a467201c02ebf5b5068dea2daa
8b0ccc2fc70aa9299156b90e517d16bd
22c976ed6dc32b760bff68a9297749d6
47cca8a6213378040e4d006bc1d1b171
130a9a72202882983b3d95408a55ca12
6f23b86ef83f71b7d32fc1e79431cc62
b2b0f2d3b47560966ad7f6773b5d6c62
022e3ff90c49987df63576ee9d047219
6042a94c3eabfaa207224b7fac32bdf5
0619c6f8d2767a970d1a189a909a312b
f6f6883cdb347fa5bad5c3cb32891b8a
a262e9faa70a24bfdb49c21e53e468eb
f90abd04e79add11247e619ed19b8770
b2ebc31f9c485f5a1f041cc9f56a5015
eb73c4514e15dea3600996a793f2598a
ed80f3c7ed005af0f3cbd4ae238c6ec0
eda98ede61f596a3

...

Title: Extracting ecospold2 files:
  Started: 05/13/2024 15:13:28
  Finished: 05/13/2024 15:15:58
  Total time elapsed: 00:02:30
  CPU %: 98.70
  Memory %: 17.03
Extracted 18121 datasets in 150.66 seconds
Applying strategy: normalize_units
Applying strategy: update_ecoinvent_locations
Applying strategy: remove_zero_amount_coproducts
Applying strategy: remove_zero_amount_inputs_with_no_activity
Applying strategy: remove_unnamed_parameters
Applying strategy: es2_assign_only_product_with_amount_as_reference_product
Applying strategy: assign_single_product_as_activity
Applying strategy: create_composite_code
Applying strategy: drop_unspecified_subcategories
Applying strategy: fix_ecoinvent_flows_pre35
Applying strategy: drop_temporary_outdated_biosphere_flows
Applying strategy: link_biosphere_by_flow_uuid
Applying strategy: link_internal_technosphere_by_composite_code
Applying strategy: delete_exchanges_missing_activity
Applying strategy: delete_ghost_exchanges
Applying strategy: remove_un

Writing activities to SQLite3 database:


18121 datasets
615644 exchanges
0 unlinked exchanges
  


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:25


Title: Writing activities to SQLite3 database:
  Started: 05/13/2024 15:16:03
  Finished: 05/13/2024 15:16:28
  Total time elapsed: 00:00:25
  CPU %: 99.90
  Memory %: 17.42
Created database: ecoinvent_3.6_cutoff

biosphere3 and ecoinvent background databases imported.

Extraction of the Ecoinvent dataset references: 57/58 unique datasets extracted from the Excel file.

Activity: S1A1Cultivation
---------

Activity S1A1Cultivation created in the foreground database.

Exchange: Production
The production exchange is arbitrary. Each activity produces itself.
Done!

Exchange: broth
Biomass flow: ref_flow
Not included in the LCA model.
Pass!

Exchange: sodium_bicarbonate
The activity sodium_bicarbonate is retrieved from the ecoinvent database.
Name: market for sodium bicarbonate
Location: GLO
Unit: kilogram
Database name: ecoinvent_3.6_cutoff
Done!

Exchange: potassium_nitrate
The activity potassium_nitrate is retrieved from the ecoinvent database.
Name: market for potassium nitrate
Locatio

The foreground databases can now be used to perform the LCA study of the SpiralG biorefinery! (see Jupyter Notebook *NB_eLCA_calc*). The name of the databases will be automatically associated with a unique identifier e.g. "*db_S1tech_1.fcd27b66eba41d0a99a84503af795f18.bw2package*". For, now, it is required to remove the code "*fcd27b66eba41d0a99a84503af795f18*" manually before running the LCA. 

The foreground database can be opened with the [Activity Browser](https://github.com/LCA-ActivityBrowser/activity-browser) to check that the data were correctly generated. 